In [1]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import argparse
import multiprocessing

In [2]:
# Import custom modules
from pretrained_models import pretrained_model
from utils import *

In [3]:
def main(args):
    # Wandb details
    user = args.wandb_entity
    project = args.wandb_project
    display_name = "test_run"

    if args.wandb_login:
        wandb.init(entity=user, project=project, name=display_name) # Initialize wandb experiment. 
        # config = wandb.config
    
    # Define dataset paths
    train_dir = args.data_directory+'/train'
    test_dir = args.data_directory+'/val'

    # Set hyperparameters
    learning_rate = args.learning_rate 
    batch_size = args.batch_size
    max_epochs = args.epochs

    # Device selection order CUDA > MPS > CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Using device: {device}")
    num_workers = max(1, multiprocessing.cpu_count() - 2) if torch.cuda.is_available() else 4 if torch.backends.mps.is_available() else 1

    use_augmentation = args.use_augmentation

    # Get dataloaders for train, val and test
    train_loader, val_loader, test_loader = dataset_split(train_dir, test_dir, batch_size=batch_size, num_workers=num_workers, augmentation=use_augmentation)

    # Load a pretrained
    model_type = args.model_type # Options = ["ResNet18", "ResNet50", "GoogLeNet", "VGG", "InceptionV3", "EfficientNetV2", "VisionTransformer"]
    model = pretrained_model(model_type, k=args.trainable_layers)

    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss() # loss function
    optimizer = get_optimizer(args.optimizer, model, learning_rate=learning_rate, weight_decay=args.weight_decay) # optimizer
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3) # learning rate scheduler

    # Print Training information
    print(f"Training with {model_type}")
    train_loop(train_loader, val_loader, model, loss_fn, optimizer, scheduler=scheduler, device=device, max_epochs=max_epochs, patience_stop=10, wandb_log=args.wandb_login)
    # class_names = ["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi", "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]
    # test_loop(test_loader, model, loss_fn, device, class_names)

    if args.wandb_login:
        wandb.finish() # End wandb experiment

In [4]:
from argparse import Namespace

from argparse import Namespace

args = Namespace(
    wandb_project="DA6401_Assignment_2",
    wandb_entity="ee20d201-indian-institute-of-technology-madras",
    data_directory="../../inaturalist_12K",  # Update this path as needed
    epochs=20,
    batch_size=32,
    model_type="EfficientNetV2",  # Options: ResNet18, ResNet50, GoogLeNet, VGG, InceptionV3, EfficientNetV2, VisionTransformer
    trainable_layers=1,
    use_augmentation=True,
    padding=1,
    optimizer="sgd",  # Options: sgd, adam
    learning_rate=0.000281,
    weight_decay=0.000827,
    wandb_login=False
)

main(args)

Using device: mps
Training with EfficientNetV2


Training Progress:   0%|          | 0/20 [00:00<?, ?epoch/s]

Epoch [1/20] → Train Loss: 2.0416, Train Acc: 37.80% | Val Loss: 1.6645, Val Acc: 66.33%

Current Learning Rate: 0.000281


Training Progress:   5%|▌         | 1/20 [03:57<1:15:05, 237.15s/epoch]

Epoch [2/20] → Train Loss: 1.2385, Train Acc: 68.65% | Val Loss: 0.8749, Val Acc: 77.39%

Current Learning Rate: 0.000281


Training Progress:  10%|█         | 2/20 [07:56<1:11:31, 238.39s/epoch]

Epoch [3/20] → Train Loss: 0.8274, Train Acc: 75.55% | Val Loss: 0.6572, Val Acc: 80.84%

Current Learning Rate: 0.000281


Training Progress:  15%|█▌        | 3/20 [11:54<1:07:32, 238.38s/epoch]